# Pinecone - Primeros pasos

1. [Introducción](#intro)
2. [Trabajando con índices](#index)
3. [Funciones de búsqueda en vectores](#query)

## Introducción <a name="intro"></a>

Lo primero que debemos realizar es crear una **API-Key** sobre la que poder auntenticarnos y, crear un proyecto sobre el que subir nuestros índices.

Por lo tanto, definimos la información necesaria:
+ **PINECONE_API_KEY**
+ **PINECONE_ENV**
+ **INDEX_NAME**

IMPORTANTE: Para poder trabajar con Pinecone desde Python debemos instalar el paquete que nos da acceso al cliente en la nube https://docs.pinecone.io/docs/python-client

In [ ]:
# !pip3 install pinecone-client

In [1]:
PINECONE_API_KEY = ""
PINECONE_ENV     = "gcp-starter"
INDEX_NAME       = 'demo-rag'

Una vez definidas nuestras variables constantes para poder interactuar con Pinecone, debemos lanzar el cliente, esto, se consigue a través de la función `pinecone.init()` esta función recibe principalmente dos parámetros:
* API-Key
* ID del entorno en la nube

De esta forma, ya hemos conectado el notebook al servidor de Pinecone

In [2]:
# Importamos Pinecone
import pinecone

# Lanzamos el servidor
pinecone.init( 
    api_key      = PINECONE_API_KEY,
    environment  = PINECONE_ENV
)

C:\Users\pere_\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Posteriormente, una vez conectados al servidor, podemos acceder a nuestros índices mediante la función `pinecone.list_indexes()` 

In [3]:
pinecone.list_indexes()

['demo-rag']

También es posible crear un índice desde cero, todo desde el cliente en Python 

`import pinecone`

`pinecone.init(api_key="YOUR_API_KEY", environment="YOUR_ENVIRONMENT")`

`pinecone.create_index("example-index", dimension=1024)`

Además de poder acceder a su nombre, para controlar el estado del mismo es posible acceder a información como
* Status del índice
* Nombre del índice
* Tamaño del índice

Esto se consigue desde la función `pinecone.describe_index()`

In [4]:
pinecone.describe_index(INDEX_NAME) # Devuelve un índice de valores

IndexDescription(name='demo-rag', metric='cosine', replicas=1, dimension=384.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [7]:
pinecone.describe_index(INDEX_NAME).dimension

384.0

## Trabajando con Índices <a name="index"></a>

Antes de crear vectores propios con archivos PDF (el objetivo central de nuestro asistente conversacional), vamos a crear algunos vectores para subirlos a nuestro índice.

Como tal, ahora mismo nuestro índice tiene una dimensionalidad de 384, vamos a crear algunos vectores desde numpy para subir a nuestro índice de Pinecone (serían posibles números de Word Embeddings)

In [8]:
import numpy as np

vec_1 = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()
vec_2 = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

In [9]:
vec_1[:3]

[1.0798374404461006, -0.3330418372229856, 0.5627409118823776]

Para poder subir estos dos vectores a Pinecone, debemos primero conectarnos al índice, esto se hace desde la función `pinecone.Index()`

In [10]:
index = pinecone.Index(INDEX_NAME)

Una vez que estemos conectados a un índice ya podremos realizar operaciones como:
* Upsert - Insertar vectores
* Query  - Búsqueda por similaridad
* Fetch  - Búsqueda de metadatos
* Delete - Borrar del índice
* Update - Actualizar

https://docs.pinecone.io/reference/query

Para subir vectores a un índice emplearemos la función `index.upsert()` primordialmente, esta función recibirá los siguientes parámetros:
* vectors: Lista de vectores

Esta lista de vectores, comúmente vendrá acompañada de campos que suelen introducirse como diccionario de datos (uno por cada vector) en donde se definen los siguientes campos clave:
+ id 
+ values : vector
+ metadata : Valores de metadatos de cada vector

In [11]:
upsert_vectors = index.upsert(  # Insertamos vectores
    vectors = [
        
        {"id"       :"vec_1", 
         "values"   : vec_1,
         "metadata" : {"contenido" : "aprendizaje supervisado"}},
        
        {"id"       :"vec_2", 
         "values"   : vec_2,
         "metadata" : {"contenido" : "aprendizaje NO supervisado"}}        
    ]
)

### Ejercicio 1

Vamos a subir 100 vectores, a la colección, al igual que en el ejemplo que acabamos de ver, para el campo metadata, vamos a tomar los posibles valores en el campo "contenido".
+ Las herramientas del científico de datos
+ Machine Learning
+ Estadística para el Data Science

El valor de "contenido" será aleatorio

#### Solución

In [12]:
import numpy as np
import random

# Generamos 100 vectores
vectores = []
for i in range(100):
    vec = np.random.normal(size=int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()
    
    # Escogemos un valor aleatorio para el campo "contenido"
    contenido = random.choice([
        "Las herramientas del científico de datos",
        "Machine Learning",
        "Estadística para el Data Science"
    ])
    
    # Creamos el diccionario para el vector con su metadata
    vector_dict = {
        "id": f"vec_{i+1}",
        "values": vec,
        "metadata": {"contenido": contenido}
    }
    
    vectores.append(vector_dict)

# Insertamos los vectores en Pinecone
upsert_vectors = index.upsert(vectors=vectores)


## Funciones de búsqueda en vectores <a name="query"></a>

Por el momento, nuestras _querys_ serán sin lenguaje natural, ya que, aún no hemos subido ningún archivo de texto, nuestros vectores no son ni más ni menos que números que siguen una distribución normal, pero, este es el objetivo, cuando llegue una preguna en lenguaje natural, se convertirá en Embeddings y, se buscarán vectores por similaridad.

Para realizar una búsqueda por similaridad en nuestro índice sobre los vectores, podemos hacer uso de la función `index.query()` la cuál recibe otro vector como parámetro y, el parámetro `top_k` que es el número de resultados a desplegar.

In [13]:
query_vector = np.random.normal(size = \
                                int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

In [14]:
ask_query = index.query(
    vector = query_vector, # Vector de búsqueda,
    top_k  = 1 # Solo el resultado con la similaridad más alta
)

El resultado almacenado en esta variable será simplemente los id de los vectores con la similariadad más alta, tantos como le hayamos pasado al parámetro `top_k`

In [15]:
ask_query

{'matches': [{'id': 'vec_11', 'score': 0.118170179, 'values': []}],
 'namespace': ''}

También es posible buscar mediante metadatos. https://docs.pinecone.io/docs/metadata-filtering

In [16]:
index.upsert([
    ("A", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
     {"genre": "comedy", "year": 2020}),
    
    ("B", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(),
     {"genre": "documentary", "year": 2019}),
    
    ("C", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(),
     {"genre": "comedy", "year": 2019}),
    
    ("D", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
     {"genre": "drama"}),
    
    ("E", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
     {"genre": "drama"})
])

{'upserted_count': 5}

In [18]:
index.query(
    vector=np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(),
    filter={
        "genre": {"$eq": "documentary"},
        "year": 2019
    },
    top_k=1,
    include_metadata=True
)

{'matches': [{'id': 'B',
              'metadata': {'genre': 'documentary', 'year': 2019.0},
              'score': -0.0687515,
              'values': []}],
 'namespace': ''}

Para borrar vectores, se puede emplear la función `index.delete()` Es posible eliminar directamente IDs concretos sobre el campo `ids` o borrar por metadatos sobre el campo `filter`

In [19]:
index.delete(ids=["A", "B"])

{}

In [20]:
index.delete(
    filter={
        "genre": {"$eq": "documentary"},
        "year": 2019
    }
)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '116', 'date': 'Tue, 14 Nov 2023 17:46:51 GMT', 'x-envoy-upstream-service-time': '17', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":3,"message":"Filters in this operation are not supported by the current index type 'Starter'.","details":[]}


In [21]:
# borrar todos los vectores
index.delete(delete_all=True)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '99', 'date': 'Tue, 14 Nov 2023 17:48:11 GMT', 'x-envoy-upstream-service-time': '20', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":3,"message":"deleteAll is not supported by the current index type 'Starter'.","details":[]}


### Ejercicio 2

Genera de nuevo todos los mismos vectores que el ejercicio 2, posteriormente, borra todos registros que sean únicamente {"contenido" : "Las herramientas del científico de datos"}

#### Solución


Para actualizar un vector se puede realizar desde la función `index.update()` https://docs.pinecone.io/docs/manage-data#updating-records

In [22]:
vec_test = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

upsert_vectors = index.upsert(  # Insertamos vectores
    vectors = [
        
        {"id"       :"test", 
         "values"   : vec_test,
         "metadata" : {"contenido" : "aprendizaje supervisado"}},    
    ]
)

In [23]:
# Actualizamos el vector
index.update(id="test", 
             values= np.ones(int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
             set_metadata={"contenido": "Estadística"})

{}